In [30]:
import tensorflow as tf

(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_valid, y_valid = X_train_full[:5000], y_train_full[:5000]
X_train, y_train = X_train_full[5000:], y_train_full[5000:]

In [31]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(55000, 28, 28)
(55000,)
(5000, 28, 28)
(5000,)


In [32]:
%%writefile tensors.proto
syntax = "proto3";

message BytesList { repeated bytes value = 1; }
message Int64List { repeated int64 value = 1 [packed = true]; }
message Feature {
    oneof kind {
        BytesList bytes_list = 1;
        Int64List int64_list = 3;
    }
};
message Features { map<string, Feature> feature = 1; };
message Example { Features features = 1; };

Overwriting tensors.proto


In [33]:
!protoc tensors.proto --python_out=. --descriptor_set_out=tensors.desc --include_imports

In [34]:
from tensors_pb2 import BytesList, Int64List
from tensors_pb2 import Feature, Features, Example

def create_example(image, label):
    image_data = tf.io.serialize_tensor(image)

    return Example(
        features=Features(
            feature={
                "image": Feature(bytes_list=BytesList(value=[image_data.numpy()])),
                "label": Feature(int64_list=Int64List(value=[label])),
            }
        )
    )

In [35]:
def write_record_new_version(X, y, name, num_files=2):
    records_per_files = len(X) // num_files
    reminder = len(X) % num_files
    paths = []
    for i in range(num_files):
        file_name = f"{name}_{i + 1}.tfrecord"
        paths.append(file_name)

        start_indx = i * records_per_files
        end_indx = (i + 1) * records_per_files if i < num_files - 1 else None

        if reminder > 0 and i >= num_files - reminder:
            end_indx += 1

        X_i = X[start_indx:end_indx]
        y_i = y[start_indx:end_indx]

        dataset = tf.data.Dataset.from_tensor_slices((X_i, y_i))
        if name == "train":
            dataset = dataset.shuffle(len(X_i), seed=42)

        with tf.io.TFRecordWriter(file_name) as writer:
            for image, label in dataset:
                example = create_example(image, label)
                writer.write(example.SerializeToString())
    return paths

In [36]:
train_path = write_record_new_version(X_train,y_train,'train')
valid_path =  write_record_new_version(X_valid,y_valid,'valid')
test_path =  write_record_new_version(X_test,y_test,'test')

In [37]:
print(train_path)
print(valid_path)
print(test_path)

['train_1.tfrecord', 'train_2.tfrecord']
['valid_1.tfrecord', 'valid_2.tfrecord']
['test_1.tfrecord', 'test_2.tfrecord']


In [38]:
def preprocess(tfrecord):
    feature_descriptions = {
        "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "label": tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    example = tf.io.parse_single_example(tfrecord, feature_descriptions)
    image = tf.io.parse_tensor(example["image"], out_type=tf.uint8)
    image = tf.reshape(image, shape=[28, 28])
    return image, example["label"]

def mnist_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None,
                  n_parse_threads=5, batch_size=32, cache=True):
    dataset = tf.data.TFRecordDataset(filepaths,
                                      num_parallel_reads=n_read_threads)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [39]:
train_set = mnist_dataset(train_path, shuffle_buffer_size=60000)
valid_set = mnist_dataset(valid_path)
test_set = mnist_dataset(test_path)

In [40]:
import numpy as np

standardize = tf.keras.layers.Normalization(input_shape=[28,28])
sample_image_batches = train_set.take(100).map(lambda image, label: image)
sample_images = np.concatenate(list(sample_image_batches.as_numpy_iterator()),
                               axis=0).astype(np.float32)
standardize.adapt(sample_images)

model = tf.keras.models.Sequential([
    standardize,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam", 
              metrics=["accuracy"])

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 28, 28)            57        
 tion)                                                           
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79567 (310.81 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 57 (232.00 Byte)
_________________________________________________________________


In [42]:
from datetime import datetime
from pathlib import Path

logs = Path() / "my_logs" / "run_" / datetime.now().strftime("%Y%m%d_%H%M%S")

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch=10)

model.fit(train_set, epochs=5, validation_data=valid_set,
          callbacks=[tensorboard_cb])

Epoch 1/5
1719/1719 [==============================] - 16s 7ms/step - loss: 5810.9912 - accuracy: 0.9263 - val_loss: 21253.4746 - val_accuracy: 0.9580
Epoch 2/5
1719/1719 [==============================] - 9s 5ms/step - loss: 1156.2948 - accuracy: 0.9669 - val_loss: 23843.6406 - val_accuracy: 0.9660
Epoch 3/5
1719/1719 [==============================] - 10s 6ms/step - loss: 2169.1736 - accuracy: 0.9771 - val_loss: 21175.2207 - val_accuracy: 0.9712
Epoch 4/5
1719/1719 [==============================] - 10s 5ms/step - loss: 65.4290 - accuracy: 0.9834 - val_loss: 25475.5801 - val_accuracy: 0.9704
Epoch 5/5
1719/1719 [==============================] - 8s 5ms/step - loss: 1767.8092 - accuracy: 0.9863 - val_loss: 20648.5879 - val_accuracy: 0.9706


In [43]:
from pathlib import Path

root = "https://ai.stanford.edu/~amaas/data/sentiment/"
filename = "aclImdb_v1.tar.gz"
filepath = tf.keras.utils.get_file(filename, root + filename, extract=True,
                                   cache_dir=".")
path = Path(filepath).with_name("aclImdb")
path

WindowsPath('datasets/aclImdb')

In [44]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")

In [45]:
len(train_pos), len(train_neg), len(test_valid_pos), len(test_valid_neg)

(12500, 12500, 12500, 12500)

In [46]:
np.random.shuffle(test_valid_pos)

test_pos = test_valid_pos[:5000]
test_neg = test_valid_neg[:5000]
valid_pos = test_valid_pos[5000:]
valid_neg = test_valid_neg[5000:]

In [47]:
def imdb_dataset(filepaths_positive, filepaths_negative): # first way of reading data from files
    reviews = []
    labels = []
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            with open(filepath) as review_file:
                reviews.append(review_file.read())
            labels.append(label)
    return tf.data.Dataset.from_tensor_slices(
        (tf.constant(reviews), tf.constant(labels)))

In [52]:
def imdb_dataset(filepaths_positive, filepaths_negative, n_read_threads=5): # second way of reading data from files
    dataset_neg = tf.data.TextLineDataset(filepaths_negative,
                                          num_parallel_reads=n_read_threads)
    dataset_neg = dataset_neg.map(lambda review: (review, 0))
    dataset_pos = tf.data.TextLineDataset(filepaths_positive,
                                          num_parallel_reads=n_read_threads)
    dataset_pos = dataset_pos.map(lambda review: (review, 1))
    return tf.data.Dataset.concatenate(dataset_pos, dataset_neg)

In [60]:
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000, seed=42)
train_set = train_set.batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)

In [61]:
max_tokens = 1000
sample_reviews = train_set.map(lambda review, label: review)
text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens, output_mode="tf_idf")
text_vectorization.adapt(sample_reviews)

In [62]:
text_vectorization.get_vocabulary()[:10]

['[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i']

In [63]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vectorization,
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
    780/Unknown - 15s 12ms/step - loss: 0.4258 - accuracy: 0.8239

In [1]:
def compute_mean_embedding(inputs):
    not_pad = tf.math.count_nonzero(inputs, axis=-1)
    n_words = tf.math.count_nonzero(not_pad, axis=-1, keepdims=True)    
    sqrt_n_words = tf.math.sqrt(tf.cast(n_words, tf.float32))
    return tf.reduce_sum(inputs, axis=1) / sqrt_n_words

another_example = tf.constant([[[1., 2., 3.], [4., 5., 0.], [0., 0., 0.]],
                               [[6., 0., 0.], [0., 0., 0.], [0., 0., 0.]]])
compute_mean_embedding(another_example)

NameError: name 'tf' is not defined

In [ ]:
embedding_size = 20
tf.random.set_seed(42)

text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens, output_mode="int")
text_vectorization.adapt(sample_reviews)

model = tf.keras.Sequential([
    text_vectorization,
    tf.keras.layers.Embedding(input_dim=max_tokens,
                              output_dim=embedding_size,
                              mask_zero=True), 
    tf.keras.layers.Lambda(compute_mean_embedding),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

In [ ]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="imdb_reviews")
train_set, test_set = datasets["train"], datasets["test"]